This jupyter notebook is used for data cleaning and analysis of the data of the public administration provided by Reply.
In this file you can find the following sections:

1. Data Cleaning and Preprocessing
2. Exploratory Data Analysis (EDA)
3. Some queries given to the agent, manually analyzed and compared with the results of the agent

## Data Cleaning and Preprocessing
In this section we will clean and preprocess the data to make it ready for the analysis. In particular, we transform datasets' names and columns headers. Then, we transform all the names in lowercase.
At the end, some little transofrmations, changing yes/no to 1/0, and creating age_group and income_brackets variables. 

In [2]:
import pandas as pd
import os

# from italian to english
filename_mapping = {
    "EntryAccreditoStipendi_202501.csv": "salary.csv",
    "EntryAccessoAmministrati_202501.csv": "admin_access.csv",
    "EntryPendolarismo_202501.csv": "commuters.csv",
    "EntryAmministratiPerFasciaDiReddito_202501.csv": "income_brackets.csv"
}

# columns headers
column_mappings = {
    "EntryAccreditoStipendi_202501.csv": {
        "comune_della_sede": "municipality_of_the_location",
        "amministrazione": "administration",
        "eta_min": "min_age",
        "eta_max": "max_age",
        "sesso": "gender",
        "modalita_pagamento": "payment_method",
        "numero": "number_of_salary_records"
    },
    "EntryAccessoAmministrati_202501.csv": {
        "regione_residenza_domicilio": "region_of_residence",
        "amministrazione_appartenenza": "administration",
        "sesso": "gender",
        "eta_max": "max_age",
        "eta_min": "min_age",
        "modalita_autenticazione": "authentication_method",
        "numero_occorrenze": "number_of_occurrences"
    },
    "EntryPendolarismo_202501.csv": {
        "provincia_della_sede": "province_of_the_location",
        "comune_della_sede": "municipality_of_the_location",
        "stesso_comune": "lives_and_works_same_municipality",
        "ente": "administration",
        "numero_amministrati": "number_of_commuters",
        "distanza_min_KM": "commute_distance_min_km",
        "distanza_max_KM": "commute_distance_max_km"
    },
    "EntryAmministratiPerFasciaDiReddito_202501.csv": {
        "comparto": "sector",
        "regione_residenza": "region_of_residence",
        "sesso": "gender",
        "eta_min": "min_age",
        "eta_max": "max_age",
        "aliquota_max": "max_tax_rate",
        "fascia_reddito_min": "min_income_bracket",
        "fascia_reddito_max": "max_income_bracket",
        "numerosita": "population_size"
    }
}

# Output directory
output_folder = "datasets"
os.makedirs(output_folder, exist_ok=True)

# function to add age_group if min_age and max_age are present
def add_age_group(df):
    if "min_age" in df.columns and "max_age" in df.columns:
        df["age_group"] = df["min_age"].astype(str) + "-" + df["max_age"].astype(str)
        cols = df.columns.tolist()
        idx = cols.index("max_age") + 1
        cols = cols[:idx] + ["age_group"] + [c for c in cols if c not in cols[:idx] + ["age_group"]]
        df = df[cols]
    return df


for original_file, new_file in filename_mapping.items():
    if not os.path.exists(original_file):
        print(f"⚠️ File not found: {original_file} — Skipping.")
        continue

    df = pd.read_csv(original_file)
    df.rename(columns=column_mappings[original_file], inplace=True)
    df = add_age_group(df)
    df.to_csv(os.path.join(output_folder, new_file), index=False)
    print(f"✅ Saved {new_file} to {output_folder}/")

✅ Saved salary.csv to datasets/
✅ Saved admin_access.csv to datasets/
✅ Saved commuters.csv to datasets/
✅ Saved income_brackets.csv to datasets/


In [ ]:
import pandas as pd

# Load datasets
salary = pd.read_csv("datasets/salary.csv")
admin_access = pd.read_csv("datasets/admin_access.csv")
commuters = pd.read_csv("datasets/commuters.csv")
income_brackets = pd.read_csv("datasets/income_brackets.csv")

In [ ]:
# Transform all string values to lowercase
salary = salary.applymap(lambda x: x.lower() if isinstance(x, str) else x)
admin_access = admin_access.applymap(lambda x: x.lower() if isinstance(x, str) else x)
commuters = commuters.applymap(lambda x: x.lower() if isinstance(x, str) else x)
income_brackets = income_brackets.applymap(lambda x: x.lower() if isinstance(x, str) else x)

# Save the transformed datasets
salary.to_csv("datasets/salary.csv", index=False)
admin_access.to_csv("datasets/admin_access.csv", index=False)
commuters.to_csv("datasets/commuters.csv", index=False)
income_brackets.to_csv("datasets/income_brackets.csv", index=False)

In [ ]:
# function to convert max_age to int64
def convert_max_age(df):
    df["max_age"] = pd.to_numeric(df["max_age"], errors="coerce").astype("Int64")
    return df
    
# apply the function
salary = convert_max_age(salary)
income_brackets = convert_max_age(income_brackets)
admin_access = convert_max_age(admin_access)

In [ ]:
#lives_and_works_same_municipality from si to yes
commuters["lives_and_works_same_municipality"] = commuters["lives_and_works_same_municipality"].map({
    "si": "yes",
    "no": "no"
})

In [ ]:
# Convert distance columns to numeric
commuters["distance_min_KM"] = pd.to_numeric(commuters["distance_min_KM"], errors="coerce").astype("Int64")
commuters["distance_max_KM"] = pd.to_numeric(commuters["distance_max_KM"], errors="coerce").astype("Int64")

# Rename columns to match the new naming convention
commuters.rename(columns={
    "distance_min_KM": "commute_distance_min_km",
    "distance_max_KM": "commute_distance_max_km"
}, inplace=True)

In [ ]:
#yes/no to 1/0
commuters["lives_and_works_same_municipality"] = commuters["lives_and_works_same_municipality"].map({
    "yes": 1,
    "no": 0
})

In [ ]:
# Get information about each dataset
print("Salary Dataset Info:")
print(salary.info())
print("\nAdmin Access Dataset Info:")
print(admin_access.info())
print("\nCommuters Dataset Info:")
print(commuters.info())
print("\nIncome Brackets Dataset Info:")
print(income_brackets.info())   

In [ ]:
import pandas as pd

income_df = pd.read_csv("datasets/income_brackets.csv")

# function to combine min and max income bracket
def combine_income_brackets(row):
    min_val = row["min_income_bracket"]
    max_val = row["max_income_bracket"]

    try:
        if pd.isna(max_val):
            return f"{int(min_val)}+"
        else:
            return f"{int(min_val)}–{int(max_val)}"
    except:
        return "Invalid"

# apply and save
income_df["income_bracket_range"] = income_df.apply(combine_income_brackets, axis=1)
income_df.to_csv("datasets/income_brackets_with_range.csv", index=False)

In [ ]:
file_path = 'datasets/admin_access.csv'
df = pd.read_csv(file_path)

import os

# m to 1 and f to 0
df['gender'] = df['gender'].map({'m': 1, 'f': 0})

# save the new DataFrame to a CSV file 
df.to_csv('datasets/admin_access.csv', index=False)

In [ ]:
import pandas as pd
file_path = 'datasets/income_brackets.csv'
df = pd.read_csv(file_path)

import os

# m to 1 and f to 0
df['gender'] = df['gender'].map({'m': 1, 'f': 0})   

# save the new DataFrame to a CSV file
df.to_csv('datasets/income_brackets.csv', index=False)

## Exploratory Data Analysis
In this notebook is showed some insights, patterns and distribution of the four datasets provided by Reply. The key idea is to show how data are distribution and some plots or analysis insights are useful to understand if some agent's answers are right or wrong. 



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)


datasets_info = {
    "AccessoAmministrati": "admin_access.csv",
    "AccreditoStipendi": "salary.csv",
    "AmministratiFasciaReddito": "income_brackets.csv",
    "Pendolarismo": "commuters.csv"
}

datasets = {}

# 2. Loading and overview datasets
for name, file in datasets_info.items():
    print(f"\n--- {name} ---")
    df = pd.read_csv(file)
    datasets[name] = df
    print(f"Shape: {df.shape}")
    print("Colonne:", df.columns.tolist())
    print("Tipi di dati:")
    print(df.dtypes)
    print("Prime 5 righe:")
    print(df.head(3))

# 3. missing data
for name, df in datasets.items():
    print(f"\n--- MISSING DATA: {name} ---")
    missing = df.isnull().sum()
    missing_pct = (missing / len(df)) * 100
    missing_df = pd.DataFrame({"MissingCount": missing, "MissingPercent": missing_pct})
    print(missing_df[missing_df["MissingCount"] > 0])

    # Visualization
    if missing_df["MissingCount"].sum() > 0:
        plt.figure()
        sns.heatmap(df.isnull(), cbar=False, yticklabels=False)
        plt.title(f"Missing Values Heatmap: {name}")
        plt.show()

In [ ]:
#Descriptive statistics
for name, df in datasets.items():
    print(f"\n--- DESCRIPTIVE STATISTICS: {name} ---")
    numeriche = df.select_dtypes(include=[np.number])
    categoriche = df.select_dtypes(exclude=[np.number])

    print("Numeric:")
    print(numeriche.describe())

    print("\nCategorical:")
    for col in categoriche.columns:
        print(f"\n{col} - {df[col].nunique()} categories")  
        print(df[col].value_counts().head())

In [ ]:
# Univariate analysis
for name, df in datasets.items():
    print(f"\n--- UNIVARIATE ANALYSIS: {name} ---")
    numeriche = df.select_dtypes(include=[np.number])
    categoriche = df.select_dtypes(exclude=[np.number])

    # Numeric variables
    for col in numeriche.columns:
        plt.figure()
        sns.histplot(df[col].dropna(), kde=True)
        plt.title(f"{name} - Distribuzione: {col}")
        plt.show()

        plt.figure()
        sns.boxplot(x=df[col])
        plt.title(f"{name} - Boxplot: {col}")
        plt.show()

    # Categorical variables
    for col in categoriche.columns[:5]:
        plt.figure()
        sns.countplot(data=df, x=col, order=df[col].value_counts().index)
        plt.xticks(rotation=45)
        plt.title(f"{name} - Frequenza: {col}")
        plt.show()

In [ ]:
# Outlier detection
for name, df in datasets.items():
    print(f"\n--- OUTLIER DETECTION: {name} ---")
    numeriche = df.select_dtypes(include=[np.number])
    for col in numeriche.columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        outliers = df[(df[col] < Q1 - 1.5 * IQR) | (df[col] > Q3 + 1.5 * IQR)]
        outlier_pct = (len(outliers) / len(df)) * 100
        print(f"{col}: {len(outliers)} outlier ({outlier_pct:.2f}%)")
        if len(outliers) > 0:
            plt.figure()
            sns.boxplot(x=df[col])
            plt.title(f"{name} - Outlier: {col}")
            plt.show()

In [ ]:
#What is the average max_age in the income_brackets dataset?
import pandas as pd

income_df = pd.read_csv("datasets/income_brackets.csv")
average_max_age = income_df["max_age"].mean()
print(f"Average max_age in income_brackets: {average_max_age:.2f}")

In [ ]:
#Which administrations have the highest number of commuters?

commuters = pd.read_csv("datasets/commuters.csv")
admin_totals = commuters.groupby("administration")["number_of_commuters"].sum().reset_index()

admin_totals_sorted = admin_totals.sort_values(by="number_of_commuters", ascending=False)

print(admin_totals_sorted.head(10))



# How many commuters live and work in the same municipality?
import pandas as pd

commuters = pd.read_csv("datasets/commuters.csv")
same_municipality_count = commuters[commuters["lives_and_works_same_municipality"] == 1]["number_of_commuters"].count()

print(f"Total commuters who live and work in the same municipality: {same_municipality_count}")


#Compare the average max_age per gender.
import pandas as pd

income = pd.read_csv("datasets/income_brackets.csv")
avg_max_age_by_gender = income.groupby("gender")["max_age"].mean().reset_index()
print(avg_max_age_by_gender)


#What is the average number_of_salary_records per age_group?
import pandas as pd
salary = pd.read_csv("datasets/salary.csv")
avg_salary_by_age_group = salary.groupby("age_group")["number_of_salary_records"].mean().reset_index()

print(avg_salary_by_age_group)


#How many unique payment_method values exist in the dataset?
import pandas as pd

salary = pd.read_csv("datasets/salary.csv")
unique_methods = salary["payment_method"].nunique()
method_list = salary["payment_method"].unique()

print(f"Unique payment_method values: {unique_methods}")
print("Payment methods:", method_list)

# What is the most common authentication_method?
import pandas as pd

admin = pd.read_csv("datasets/admin_access.csv")
most_common_method = admin["authentication_method"].value_counts().idxmax()
count = admin["authentication_method"].value_counts().max()

print(f"The most common authentication method is '{most_common_method}' with {count} occurrences.")



In [ ]:
salary_df = pd.read_csv("datasets/salary.csv")
avg_salary_by_age_group = salary_df.groupby("age_group")["number_of_salary_records"].mean().reset_index()

print(avg_salary_by_age_group)

import matplotlib.pyplot as plt

plt.figure(figsize=(8,5))
plt.bar(avg_salary_by_age_group["age_group"], avg_salary_by_age_group["number_of_salary_records"])
plt.xlabel("Age Group")
plt.ylabel("Average Salary Records")
plt.title("Average Salary per Age Group")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
admin_df = pd.read_csv("datasets/admin_access.csv")

plt.figure(figsize=(8, 5))
plt.hist(admin_df["max_age"].dropna(), bins=10, color='skyblue', edgecolor='black')
plt.title("Histogram of Max Age (admin_access dataset)")
plt.xlabel("Max Age")
plt.ylabel("Frequency")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
#What is the average commute distance for people who do not live and work in the same municipality?
commuters = pd.read_csv("datasets/commuters.csv")

commuters["commute_distance_min_km"] = pd.to_numeric(commuters["commute_distance_min_km"], errors="coerce")
commuters["commute_distance_max_km"] = pd.to_numeric(commuters["commute_distance_max_km"], errors="coerce")

commuters["commute_avg_km"] = commuters[["commute_distance_min_km", "commute_distance_max_km"]].mean(axis=1)

#filter commuters that do not live and work in the same municipality
filtered = commuters[commuters["lives_and_works_same_municipality"] == 0]

#calculate average distance
average_distance = filtered["commute_avg_km"].mean()

print(f"Average commute distance for people who do NOT live and work in the same municipality: {average_distance:.2f} km")

In [ ]:
import pandas as pd
commuters = pd.read_csv("datasets/commuters.csv")

#convert distance in km
distance_threshold_km = 32.19  # 20 miles

#convert distance in km
commuters["commute_distance_min_km"] = pd.to_numeric(commuters["commute_distance_min_km"], errors="coerce")

#calculate total commuters per administration
total_commuters = commuters.groupby("administration")["number_of_commuters"].sum().rename("total_commuters")

#calculate commuters that do at least 20 miles (on the minimum distance)
long_commuters = commuters[commuters["commute_distance_min_km"] >= distance_threshold_km]
long_commuters_count = long_commuters.groupby("administration")["number_of_commuters"].sum().rename("over_20_miles")

#merge total commuters and long commuters   
merged = pd.merge(total_commuters, long_commuters_count, on="administration", how="left").fillna(0)

#calculate percentage
merged["percent_over_20_miles"] = (merged["over_20_miles"] / merged["total_commuters"]) * 100

#sort by percentage
result = m  erged.sort_values(by="percent_over_20_miles", ascending=False)

#print the top 10
print(result[["percent_over_20_miles"]].head(10))
